# 일등이조 (신경민, 김영성)
# 국방일보 전체 기사 크롤링 코드 (+전처리)

In [19]:
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import requests
from urllib.request import urlopen
import time
import re

----

## 1. 셀레니움 드라이버 설정 및 크롤링할 url을 열기

In [37]:
service = Service(executable_path=ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)
url = 'https://kookbang.dema.mil.kr/newsWeb/allToday.do'
driver.get(url)

## 2. 날짜 리스트 만들기
- pandas의 date_range를 통하여 원하는 기간 리스트에 담기

In [39]:
start_date = '2019-04-08'
end_date = '2019-04-09'
date_range = pd.date_range(start=start_date, end=end_date, freq='B')
date_list = date_range.strftime('%Y.%m.%d').tolist()
print(date_list)

['2019.04.08', '2019.04.09']


## 3 제목, 연도, url주소 리스트 담기
- `document.getElementById().value = ''`로 날짜 입력창 비움
- date_list에서 날짜 꺼내 입력후 검색 버튼 클릭
- BeautifulSoup으로 전체 HTML 파싱
- div id=`TODAY_ATCE_CTGR_00{i}0000000`에서 카테고리별 기사 영역 추출
  - `name_list`: 기사 제목 (`h3.eps1` 태그에서 추출)
  - `year_list`: 기사 연도 (`<span>` 태그의 텍스트 앞부분 사용)
  - `url_list`: 기사 상세 주소 (`a` 태그의 `href` 중 `newstream` 포함된 경우만 저장)

In [41]:
name_list = []
year_list = []
url_list = []

for date in date_list:
    date_input = driver.find_element(By.ID, 'news_s_date')
    driver.execute_script("document.getElementById('news_s_date').value = '';")
    time.sleep(2)
    date_input.send_keys(date)
    time.sleep(2)

    try:
        driver.find_element(By.XPATH, '//*[@id="mainForm"]/div/button').click()
    except Exception as e:
        print("연결 실패:", e)
    time.sleep(3)

    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    for i in [1, 2, 3, 5]:
        article = soup.find_all('div', {'id': f'TODAY_ATCE_CTGR_00{i}0000000'})

        for art in article:
            cart = art.find('h3')
            articles = art.find_all('ul', {'class': 'basic_list'})
            for ul in articles:
                titles = ul.find_all('h3', {'class': 'eps1'})
                date_tags = ul.find_all('span')
                a_tags = ul.find_all('a')

                for pn in titles:
                    name_list.append(pn.text.strip() + ' <' + cart.text.strip() + '>')
                for dat in date_tags:
                    year_list.append(dat.text.strip()[:13])
                for a in a_tags:
                    href = a.get('href')
                    if href and href.startswith('/newsWeb/'):
                        url_list.append('https://kookbang.dema.mil.kr' + href)
                    else:
                        print(f"[경고] 잘못된 href: {href}")

## 4. 기사 본문 크롤링 및 정제
- `bad_keywords` 불필요한 문장 필터링 키워드 리스트
- `con_list` 크롤링한 기사 본문을 저장할 리스트
- url별로 기사 본문 html 파싱
- `<div id="article_body_view">` 내 텍스트 수집
- `<br>` 처리, 줄바꿈 분리, 불필요 문장 제거
- 전체 본문 공백 제거 후 리스트에 저장
- 에러 발생 시 '에러 발생'으로 처리

In [55]:
bad_keywords = ['페이스북', '트위터', '밴드', '카카오톡', 'url복사', '기자', '글=', '사진']
con_list = []

for i, url in enumerate(url_list):
    try:
        print(f"({i}/{len(url_list)}) 크롤링 중")
        soup = BeautifulSoup(urlopen(url), 'html.parser')
        contents = soup.find('div', {'id': 'article_body_view'})

        if not contents:
            con_list.append('본문 없음')
            continue

        for br in contents.find_all('br'):
            br.replace_with('\n')

        full_text = contents.get_text(separator='\n').strip()
        lines = [line.strip() for line in full_text.split('\n') if line.strip()]
        clean_lines = [line for line in lines if not any(bad in line for bad in bad_keywords)]

        final_text = re.sub(r'\s+', '', ''.join(clean_lines)).strip()
        con_list.append(final_text)
        print("완료")
    except Exception as e:
        print(f"에러 발생: {e}")
        con_list.append('에러 발생')

(0/124) 크롤링 중: https://kookbang.dema.mil.kr/newsWeb/20190408/31/ATCE_CTGR_0010010000/view.do
완료
(1/124) 크롤링 중: https://kookbang.dema.mil.kr/newsWeb/20190408/43/ATCE_CTGR_0010010000/view.do
완료
(2/124) 크롤링 중: https://kookbang.dema.mil.kr/newsWeb/20190408/5/ATCE_CTGR_0010010000/view.do
완료
(3/124) 크롤링 중: https://kookbang.dema.mil.kr/newsWeb/20190408/4/ATCE_CTGR_0010010000/view.do
완료
(4/124) 크롤링 중: https://kookbang.dema.mil.kr/newsWeb/20190408/29/ATCE_CTGR_0010010000/view.do
완료
(5/124) 크롤링 중: https://kookbang.dema.mil.kr/newsWeb/20190408/8/ATCE_CTGR_0010010000/view.do
완료
(6/124) 크롤링 중: https://kookbang.dema.mil.kr/newsWeb/20190408/6/ATCE_CTGR_0010010000/view.do
완료
(7/124) 크롤링 중: https://kookbang.dema.mil.kr/newsWeb/20190408/12/ATCE_CTGR_0010010000/view.do
완료
(8/124) 크롤링 중: https://kookbang.dema.mil.kr/newsWeb/20190408/11/ATCE_CTGR_0010010000/view.do
완료
(9/124) 크롤링 중: https://kookbang.dema.mil.kr/newsWeb/20190408/14/ATCE_CTGR_0010010000/view.do
완료
(10/124) 크롤링 중: https://kookbang.dema.mil.kr

## 5. DataFrame으로 변환

In [89]:
df = pd.DataFrame({
    'Date': year_list,
    'Title': name_list,
    'content':con_list
})
df

,Date,Title,content
0,2019. 04. 05.,장군단 미래안보전략과정 개설 <국방>,"국방대, 급변하는 안보정세 속 현안 교육 국방대학교 박영준 교수부장이 지난 4일 육..."
1,2019. 04. 08.,"국립서울현충원, 호국문예백일장·그림그리기 대회 개최 <국방>",8일까지 홈페이지에서 신청…최우수작 국방부장관상 국립서울현충원은 순국선열과 호국영령...
2,2019. 04. 08.,"[종합] 군, 대장급 인사 단행…육군참모총장 서욱·공군참모총장 원인철·해병대사령관 ...","국방부는 8일 ""정부는 신임 육·공군참모총장과 한미연합군사령부 부사령관, 지상작전사..."
3,2019. 04. 08.,"[종합2보] 군, 대장급 인사 단행…육군참모총장 서욱·공군참모총장 원인철·해병대사령...","연합사 부사령관 최병혁, 지작사령관 남영신 군, 대장급 인사 단행…육군참모총장 서욱..."
4,2019. 04. 05.,유엔사 “DMZ 둘레길 긍정적 <국방>,매일 한국 정부·군과 협의” 민간 관광객 안전 최우선 고려 유엔사는 “둘레길 조성사...
...,...,...,...
1290,2019. 05. 08.,상실감을 자신감으로! <오피니언>,박지웅 중사 육군25사단 감악산대대 특전사·특공·수색대 등 특수한 임무를 수행하는 ...
1291,2019. 05. 08.,"사랑하는 대한민국, 사랑하는 내 가족 <오피니언>",최승주 상사 육군특수전사령부 국제평화지원단 올해 5월로 군 생활을 시작한 지 10년...
1292,2019. 05. 08.,[윤지원 기고] 신북방정책 구현과 한·러 국방협력 <오피니언>,-모스크바 국제안보회의를 다녀와서 윤 지 원 상명대 국가안보학과 교수 2012년부터...
1293,2019. 05. 08.,[신기진 국방광장] 특별법 제정으로 ‘6·25무공훈장 찾아주기’ 전개 <오피니언>,신 기 진 육군인사사령부·대령 정부는 지난 4월 23일에 ‘6·25전쟁 무공훈장 수...


## 6. DataFrame으로 변환한걸 csv파일로 저장

In [91]:
df.to_csv('crawl_data/13_한국기사_전체.csv', index=False, encoding='utf-8-sig')